<h2>All imports</h2>

In [8]:
import numpy as np
import pandas as pd
import string
import time
import json
import re
import polars as pl
from collections import defaultdict
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

<h2>Reading data from CSV into a pandas data frame</h2>

In [15]:
corpus = pd.read_csv("Dataset\india-news-headlines.csv", encoding='utf-8')

<h2>Converting the pandas dataframe into a polars dataframe to increase efficiency of compuatations</h2>
<font color='grey'>"Polars supports more parallel operations than Pandas. As Polars is written in Rust, it can run many operations in parallel. Polars supports lazy evaluation. Polars examines your queries, optimizes them, and looks for ways to accelerate the query or reduce memory usage."</font>

In [16]:
df = pl.from_pandas(corpus)
display(df.head())
#display(df.describe())

publish_date,headline_category,headline_text
i64,str,str
20010102,"""unknown""","""Status quo wil..."
20010102,"""unknown""","""Fissures in Hu..."
20010102,"""unknown""","""America's unwa..."
20010102,"""unknown""","""For bigwigs; i..."
20010102,"""unknown""","""Extra buses to..."


<h2>Pre processing of corpus</h2>
<font color='grey'>Tokenizing, removing stop words, stemming or lemmatizing the words, and possibly other techniques like removing punctuation or normalizing capitalization.</font>

In [25]:
#nltk.download('all')

# Tokenization
corpus['headline_text'] = corpus['headline_text'].apply(nltk.word_tokenize)
corpus['headline_text']


In [ ]:
# Lowercasing
df['headline_text'] = df['headline_text'].apply(lambda x: [word.lower() for word in x])